In [1]:
from matplotlib import pylab
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from salishsea_tools import tidetools
from salishsea_tools import (nc_tools,viz_tools)
import matplotlib.cm as cm
from matplotlib import animation
from numpy import *
from pylab import *
%matplotlib inline

In [2]:
mesh =nc.Dataset('/data/jieliu/MEOPAR/SalishSea/results/2hr_mesh_deepen/mesh_mask.nc')
nc_tools.show_variables(mesh)

[u'e1f', u'e1t', u'e1u', u'e1v', u'e2f', u'e2t', u'e2u', u'e2v', u'e3t', u'e3t_0', u'e3u', u'e3v', u'e3w', u'e3w_0', u'ff', u'fmask', u'fmaskutil', u'gdept', u'gdept_0', u'gdepu', u'gdepv', u'gdepw', u'gdepw_0', u'glamf', u'glamt', u'glamu', u'glamv', u'gphif', u'gphit', u'gphiu', u'gphiv', u'mbathy', u'nav_lat', u'nav_lev', u'nav_lon', u'time_counter', u'tmask', u'tmaskutil', u'umask', u'umaskutil', u'vmask', u'vmaskutil']


In [3]:
## extract the variables that Ariane wants
gglamt = mesh.variables['glamt'][0]
gglamu = mesh.variables['glamu'][0]
ggphit = mesh.variables['gphit'][0]
ggphiv = mesh.variables['gphiv'][0]
ggdepw = mesh.variables['gdepw'][0]

ee2u = mesh.variables['e2u'][0]
ee1v = mesh.variables['e1v'][0]
ee1t = mesh.variables['e1t'][0]
ee2t = mesh.variables['e2t'][0]
ee3t = mesh.variables['e3t'][0]
ttmask = mesh.variables['tmask'][0]

In [4]:
## create new shape of these variables or use np.empty_like()
glamt_new = np.zeros([ 170, 138])
glamu_new = np.zeros([ 170, 138])
gphit_new = np.zeros([ 170, 138])
gphiv_new = np.zeros([ 170, 138])
gdepw_new = np.zeros([40, 170, 138])

e2u_new = np.zeros([ 170, 138])
e1v_new = np.zeros([ 170, 138])
e1t_new = np.zeros([ 170, 138])
e2t_new = np.zeros([ 170, 138])
e3t_new = np.zeros([ 40, 170, 138])
tmask_new = np.zeros([ 40, 170, 138])

In [5]:
## assign values of corresponding indices
for j in np.arange(170) :
    for i in np.arange(138) :
        glamt_new[ j, i] = gglamt[  349 + j, 259 + i]
        glamu_new[ j, i] = gglamu[ 349 + j, 259 + i]
        gphit_new[ j, i] = ggphit[ 349 + j, 259 + i]
        gphiv_new[ j, i] = ggphiv[ 349 + j, 259 + i]
        gdepw_new[ :, j, i] = ggdepw[:, 349 + j, 259 + i]
        e2u_new[ j, i] = ee2u[ 349 + j, 259 + i]
        e1v_new[ j, i] = ee1v[ 349 + j, 259 + i]
        e1t_new[ j, i] = ee1t[ 349 + j, 259 + i]
        e2t_new[ j, i] = ee2t[ 349 + j, 259 + i]
        e3t_new[ :, j, i] = ee3t[ :, 349 + j, 259 + i]
        tmask_new[:, j, i] = ttmask[:,349 + j, 259 + i] ##ttmask shape four dimensions

In [6]:
tmask_new.shape

(40, 170, 138)

In [12]:
# build mesh_new nc file
mesh_new = nc.Dataset('mesh_mask_new.nc', 'w')
#nc_tools.init_dataset_attrs(
    #grid_V, 
    #title='Salinity Temperature Initial Conditions based on Nowcast June 14, 2015 for extended Fraser river channel', 
    #notebook_name='Explore1h_plumeFileStructure', 
    #nc_filepath='/data/jieliu/MEOPAR/river-treatment/14nor_NW/1h_plume_06160629_grid_T.nc')
    #comment='Combined grid_T file with extended Fraser bathymetry from June 16, to 29')
mesh_new.createDimension('y', 170)
mesh_new.createDimension('x', 138)
mesh_new.createDimension('depth',size = 40)
mesh_new.createDimension('time_counter', None)
#grid_V.createDimension('tbnds', size = 2)
nc_tools.show_dimensions(mesh_new)
# show variables
#nav_lat = grid_V.createVariable('nav_lat', 'float32', ('y','x'))
#nav_lat.long_name = 'Latitude'
#nav_lat.units = 'degrees_north'
#nav_lat[:] = latv
#nav_lon = grid_V.createVariable('nav_lon', 'float32', ('y','x'))
#nav_lon.long_name = 'Longitude'
#nav_lon.units = 'degrees_east'
#nav_lon[:] = lonv
#depths = mesh_new.createVariable('depthv', 'float32', ('depthv'), zlib=True)
#depths = foo.createVariable('Bathymetry', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

#depthv.long_name = 'Depth'
#depthv.coordinates = 'depthV'
#depthv.units = 'm'
#depthv.positive = 'down'
#depthv.valid_range = np.array((4., 428.))##minimum depth 4m
#depthv[:] = depths

glamt = mesh_new.createVariable('glamt', 'float32', 
                               ('time_counter','y','x'))  
glamt.coordinates = 'time_counter nav_lon nav_lat'
glamt.grid = 'SalishSea6, extended and deepen Fraser'
glamt[0] = glamt_new


glamu = mesh_new.createVariable('glamu', 'float32', 
                               ('time_counter','y','x'))  
glamt.coordinates = 'time_counter nav_lon nav_lat'
glamt.grid = 'SalishSea6, extended and deepen Fraser'
glamu[0] = glamu_new

gphit = mesh_new.createVariable('gphit', 'float32', 
                               ('time_counter','y','x'))  
gphit.coordinates = 'time_counter nav_lon nav_lat'
gphit.grid = 'SalishSea6, extended and deepen Fraser'
gphit[0] = gphit_new

gphiv = mesh_new.createVariable('gphiv', 'float32', 
                               ('time_counter','y','x'))  
gphiv.coordinates = 'time_counter nav_lon nav_lat'
gphiv.grid = 'SalishSea6, extended and deepen Fraser'
gphiv[0] = gphiv_new

gdepw = mesh_new.createVariable('gdepw', 'float32', 
                               ('time_counter','depth','y','x'))  
gdepw.coordinates = 'time_counter depth nav_lon nav_lat'
gdepw.grid = 'SalishSea6, extended and deepen Fraser'
gdepw[0] = gdepw_new

e2u = mesh_new.createVariable('e2u', 'float32', 
                               ('time_counter','y','x'))  
e2u.coordinates = 'time_counter nav_lon nav_lat'
e2u.grid = 'SalishSea6, extended and deepen Fraser'
e2u[0] = e2u_new


e1v = mesh_new.createVariable('e1v', 'float32', 
                               ('time_counter','y','x'))  
e1v.coordinates = 'time_counter nav_lon nav_lat'
e1v.grid = 'SalishSea6, extended and deepen Fraser'
e1v[0] = e1v_new


e1t = mesh_new.createVariable('e1t', 'float32', 
                               ('time_counter','y','x'))  
e1t.coordinates = 'time_counter nav_lon nav_lat'
e1t.grid = 'SalishSea6, extended and deepen Fraser'
e1t[0] = e1t_new

e2t = mesh_new.createVariable('e2t', 'float32', 
                               ('time_counter','y','x'))  
e2t.coordinates = 'time_counter nav_lon nav_lat'
e2t.grid = 'SalishSea6, extended and deepen Fraser'
e2t[0] = e2t_new


e3t = mesh_new.createVariable('e3t', 'float32', 
                               ('time_counter','depth','y','x'))  
e3t.coordinates = 'time_counter depth nav_lon nav_lat'
e3t.grid = 'SalishSea6, extended and deepen Fraser'
e3t[0] = e3t_new


tmask = mesh_new.createVariable('tmask', 'float32', 
                               ('time_counter','depth','y','x'))  
tmask.coordinates = 'time_counter depth nav_lon nav_lat'
tmask.grid = 'SalishSea6, extended and deepen Fraser'
tmask[0] = tmask_new


mesh_new.history = """[2015-06-16] Created"""

<type 'netCDF4.Dimension'>: name = 'y', size = 170

<type 'netCDF4.Dimension'>: name = 'x', size = 138

<type 'netCDF4.Dimension'>: name = 'depth', size = 40

<type 'netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 0



In [13]:
mesh_new.close()

In [14]:
ab = nc.Dataset('mesh_mask_new.nc')

In [15]:
nc_tools.show_variables(ab)

[u'glamt', u'glamu', u'gphit', u'gphiv', u'gdepw', u'e2u', u'e1v', u'e1t', u'e2t', u'e3t', u'tmask']


In [16]:
abb = ab.variables['e2t']
abb.shape

(1, 170, 138)